In [ ]:
## WORKING VERSION OF LYRIC GRABBER [[USER INPUT]]

import requests
from bs4 import BeautifulSoup

def get_lyrics(song_url):
    response = requests.get(song_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all div elements with the custom attribute
        lyrics_divs = soup.find_all('div', attrs={'data-lyrics-container': 'true'})
        
        if lyrics_divs:
            lyrics_list = []
            for lyrics_div in lyrics_divs:
                lyrics = lyrics_div.get_text()
                lyrics_list.append(lyrics.strip())
            return lyrics_list
        else:
            return ["Lyrics not found with the specified attribute."]
    else:
        return ["Failed to fetch page content."]

def format_lyrics(lyrics):
    # Define a list of punctuation marks to consider
    punctuation_marks = ['.', '?', '!']
    
    # Split lines and process each line
    lines = lyrics.split('\n')
    formatted_lines = []
    
    for line in lines:
        new_line = ''
        prev_char = ''
        
        for char in line:
            if char == '[':
                new_line += '\n\n' + char
            elif char == ']':
                new_line += char + '\n'
            elif char in punctuation_marks and prev_char not in [',', "'", '"', ';', ':']:
                new_line += char + '\n'
            elif char.isalpha() and char.isupper() and prev_char.isalpha() and prev_char.islower():
                new_line += '\n' + char
            else:
                new_line += char
            prev_char = char
        
        formatted_lines.append(new_line)
    
    formatted_lyrics = '\n'.join(formatted_lines)
    return formatted_lyrics



def search_another_song():
    return input("Do you want to search for another song? (y/n): ").lower().strip() == "y"

while True:
    artist_name = input("Enter the artist's name: ")
    song_title = input("Enter the song title: ")
    
    # Replace artist_name and song_title in the URL
    song_url = f"https://genius.com/{artist_name.replace(' ', '-')}-{song_title.replace(' ', '-')}-lyrics"
    lyrics_list = get_lyrics(song_url)
    
    if lyrics_list[0]:
        merged_lyrics = ""
        for idx, lyrics in enumerate(lyrics_list):
            formatted_lyrics = format_lyrics(lyrics)
            merged_lyrics += formatted_lyrics + "\n\n"  # Separate with two new lines
        
        filename = f"{artist_name}_{song_title}.txt"
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(merged_lyrics)
            print(f"Lyrics saved to '{filename}'")
    else:
        print("Lyrics not available or not found.")
    
    if not search_another_song():
        break
